In [ ]:
#preprocessing
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
# Load the dataset
data = pd.read_csv('/content/C cube dataset.csv')

# Take first 2 columns: 'Question' and 'Answer'
questions = data['Question'].astype(str).tolist()
answers = data['Answer'].astype(str).tolist()

# Add start and end tokens to answers
answers = ["<start> " + a + " <end>" for a in answers]
# Clean text function
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9?.!,]+", " ", text)
    return text

questions = [clean_text(q) for q in questions]
answers = [clean_text(a) for a in answers]
# Tokenize questions
tokenizer_q = Tokenizer(filters='')
tokenizer_q.fit_on_texts(questions)
question_seq = tokenizer_q.texts_to_sequences(questions)
question_padded = pad_sequences(question_seq, padding='post')

# Tokenize answers
tokenizer_a = Tokenizer(filters='')
tokenizer_a.fit_on_texts(answers)
answer_seq = tokenizer_a.texts_to_sequences(answers)
answer_padded = pad_sequences(answer_seq, padding='post')
# Save vocabulary sizes
VOCAB_SIZE_Q = len(tokenizer_q.word_index) + 1
VOCAB_SIZE_A = len(tokenizer_a.word_index) + 1
print("Questions padded shape:", question_padded.shape)
print("Answers padded shape:", answer_padded.shape)
print("Vocab size (Questions):", VOCAB_SIZE_Q)
print("Vocab size (Answers):", VOCAB_SIZE_A)


Questions padded shape: (184, 10)
Answers padded shape: (184, 21)
Vocab size (Questions): 289
Vocab size (Answers): 821


In [ ]:
#Encoder, Decoder, and Seq2Seq (with GRU) — TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.models import Model
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.enc_units = enc_units
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.gru = GRU(enc_units,
                   return_sequences=True,
                   return_state=True,
                   recurrent_initializer='glorot_uniform')
    def call(self, x):
      x = self.embedding(x)
      output, state = self.gru(x)
      return output, state
class Decoder(tf.keras.Model):
   def __init__(self, vocab_size, embedding_dim, dec_units):
     super(Decoder, self).__init__()
     self.dec_units = dec_units
     self.embedding = Embedding(vocab_size, embedding_dim)
     self.gru = GRU(dec_units,
                    return_sequences=True,
                    return_state=True,
                    recurrent_initializer='glorot_uniform')
     self.fc = Dense(vocab_size)
   def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    x = self.fc(output)
    return x, state

VOCAB_SIZE = 10000  # change as per tokenizer
EMBEDDING_DIM = 256
UNITS = 512
encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, UNITS)
decoder = Decoder(VOCAB_SIZE, EMBEDDING_DIM, UNITS)








In [ ]:
#Training Loop (Teacher Forcing)
import numpy as np
import tensorflow as tf

# Define the loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none'
    )
optimizer = tf.keras.optimizers.Adam()

    # Loss function
def loss_function(real, pred):
   mask = tf.math.not_equal(real, 0)  # Ignore padding (0)
   loss_ = loss_object(real, pred)
   mask = tf.cast(mask, dtype=loss_.dtype)
   loss_ *= mask
   return tf.reduce_mean(loss_)

                        # Training step

def train_step(inp, targ, encoder, decoder, batch_size, targ_lang_tokenizer):
    loss = 0

    with tf.GradientTape() as tape:
         enc_output, enc_hidden = encoder(inp)
         dec_hidden = enc_hidden

                                                        # <start> token assumed to be at index 1
         dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']] * batch_size, 1)

         for t in range(1, targ.shape[1]):
               predictions, dec_hidden = decoder(dec_input, dec_hidden)
               loss += loss_function(targ[:, t], predictions[:, 0, :])
               dec_input = tf.expand_dims(targ[:, t], 1)  # Teacher forcing

    batch_loss = loss / int(targ.shape[1])
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the sequences to ensure equal length
input_tensor = pad_sequences(input_sequences, padding='post')
target_tensor = pad_sequences(target_sequences, padding='post')

NameError: name 'input_sequences' is not defined

In [ ]:
questions = list(data['Question'])
answers = list(data['Answer'])

# Add <start> and <end> tokens to answers
answers = ['<start> ' + ans + ' <end>' for ans in answers]
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizer for questions
input_tokenizer = Tokenizer(filters='')
input_tokenizer.fit_on_texts(questions)
input_sequences = input_tokenizer.texts_to_sequences(questions)

# Tokenizer for answers
target_tokenizer = Tokenizer(filters='')
target_tokenizer.fit_on_texts(answers)
target_sequences = target_tokenizer.texts_to_sequences(answers)
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_tensor = pad_sequences(input_sequences, padding='post')
target_tensor = pad_sequences(target_sequences, padding='post')

In [ ]:
# Define model hyperparameters
embedding_dim = 256
units = 512
BATCH_SIZE = 64

# Input and target vocab sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1
encoder = Encoder(input_vocab_size, embedding_dim, units)
decoder = Decoder(target_vocab_size, embedding_dim, units)


In [ ]:
# Assume input_tensor and target_tensor are ready from preprocessing step
# Define buffer size and batch size
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE

# Vocabulary sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Initialize encoder and decoder
encoder = Encoder(input_vocab_size, embedding_dim, units)
decoder = Decoder(target_vocab_size, embedding_dim, units)


# Training loop
EPOCHS = 20

for epoch in range(EPOCHS):
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, encoder, decoder, BATCH_SIZE, target_tokenizer)
        total_loss += batch_loss

    print(f'Epoch {epoch + 1}, Loss: {total_loss / steps_per_epoch:.4f}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:1387: UserWarning: Layer 'encoder_3' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Model Encoder does not have a `call()` method implemented.''
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'encoder_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This

NotImplementedError: Exception encountered when calling Encoder.call().

[1mModel Encoder does not have a `call()` method implemented.[0m

Arguments received by Encoder.call():
  • args=('tf.Tensor(shape=(64, 10), dtype=int32)',)
  • kwargs=<class 'inspect._empty'>